# Tutorial 2 - Hierarchical Agents

## Functionalities:
- An agent (referred as meta agent) can also assign other agents (referred as inner agents) as the function call, thereby creating a hierarchical structure
- Whenever the inner agent is called, we will run the full inner agent's process until termination
- Maximum number of steps for each agent (meta agent or inner agent) can be defined in `max_subtasks` (default: 5) variable when initialising agent

## Additional Details:
- The inner agent will take in meta agent's instruction as input
- In addition, the inner agent will also have access to meta agent's task and subtasks completed for global context
- Inner agent's output will directly go into meta agent's subtasks completed dictionary, enabling it to directly update global context

## Design Philosophy:
- Bottom layers can get context of the top layers for more context-specific generation
- Top layers choose bottom layers in a hierarchical fashion so as to limit the number of choices at each level for better performance
- `subtasks_completed` serves as the global context for the meta agents and the inner agents, and can be summarised as needed, e.g. when performing a new task to filter out irrelevant information 

# Install TaskGen

In [1]:
# !pip install taskgen-ai

In [2]:
# Set up API key and do the necessary imports
import os
from taskgen import *

os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

# Define Agents and Meta Agents
- You can also create a Meta Agent that uses other Agents (referred to as Inner Agents) as functions
- Create your Meta agent using `Agent()` (Note: No different from usual process of creating Agents - your Meta Agent is also an Agent)
- Set up an Inner Agent list and assign it to your Meta agent using `assign_agents(agent_list)`

## Example Meta Agent Setup
```python
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Takes in menu items and curates them based on profitability'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions.', max_subtasks = 2),
    Agent('Economist', 'Takes in dish names and comes up with fictitious pricing for each of them')
    ]

my_agent.assign_agents(agent_list)
```

## Run the Meta Agent
- Let us run the agent and see the interactions between the Meta Agent and Inner Agents to solve the task!
```python
output = my_agent.run('Give me 5 menu items with name, description, ingredients and price based on Italian food choices.')
```

In [3]:
def random_number_from_string(input_string):
    ''' Returns a random number between 1 and 10 when given an input string '''
    import hashlib
    import random
    # Hash the input string
    hashed_string = hashlib.sha256(input_string.encode()).hexdigest()
    
    # Convert the hashed string to an integer
    hashed_integer = int(hashed_string, 16)
    
    # Seed the random number generator with the hashed integer
    random.seed(hashed_integer)
    
    # Generate a random number between 1 and 10
    return random.randint(1, 10)

In [4]:
# assign a dish pricing function for economist
# Note: Try to make the output informative that is understandable by the Agent - use free text or json to explain the output so that someone can understand without the input
def dish_price(list_of_dish_names: list) -> dict:
    ''' Takes in <list_of_dish_names> and outputs price '''
    if not isinstance(list_of_dish_names, list):
        list_of_food_items = [list_of_dish_names]
    output = {}
    for each in list_of_dish_names:
        output[each] = '$'+str(random_number_from_string(each))
    return output
    
fp = Function(fn_description = "Takes in <list_of_dish_names: list> and outputs price", 
              output_format = {"Price": "Dish name as key and price as value, type: dict"}, 
              examples = {"list_of_dish_names": ["Pasta", "Pizza"], "Price": [{'Pasta': '$5'}, {'Pizza':'$2'}]},
              external_fn = dish_price)

In [5]:
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

In [6]:
# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Takes in menu items and curates them based on profitability'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions. Does not generate prices or ingredients.', max_subtasks = 2),
    Agent('Economist', 'Takes in dish names and comes up with fictitious pricing for each of them').assign_functions([fp])
    ]

In [7]:
my_agent.assign_agents(agent_list)

# Approach 1: Run a generic task with the Meta-Agent
- Let us run a general task to create 5 menu items for an Italian restaurant
- You can also specify how Agents interact with one another in the task, otherwise the Meta-Agent will infer what should be done

In [8]:
output = my_agent.run('Give me 5 menu items with name, description, ingredients and price based on Italian food choices.')

Subtask identified: Generate interesting dish names and descriptions for Italian food using Creative Writer.
Calling function Creative Writer with parameters {'instruction': 'Generate interesting dish names and descriptions for Italian food using Creative Writer.'}

### Start of Inner Agent: Creative Writer ###
Subtask identified: Brainstorm creative dish names for Italian food based on popular Italian dishes and ingredients.
Getting LLM to perform the following task: Brainstorm creative dish names for Italian food based on popular Italian dishes and ingredients.
> {
1. "Dish Name": "Pesto Pollo Pasta",
   "Description": "Tender chicken breast served over al dente pasta, tossed in a flavorful basil pesto sauce.",
2. "Dish Name": "Caprese Bruschetta",
   "Description": "Toasted bread topped with fresh mozzarella, juicy tomatoes, basil, and a drizzle of balsamic glaze.",
3. "Dish Name": "Ravioli di Mare",
   "Description": "Homemade ravioli stuffed with a medley of seafood, served in a l

In [9]:
output = my_agent.reply_user()

{'Menu Items': [{'Name': 'Pesto Pollo Pasta', 'Description': 'Indulge in tender chicken breast served over perfectly cooked pasta, coated in a delightful basil pesto sauce.', 'Ingredients': 'Chicken breast, pasta, basil pesto sauce', 'Price': '$3'}, {'Name': 'Caprese Bruschetta', 'Description': 'Savor the crunch of toasted bread topped with creamy mozzarella, ripe tomatoes, fresh basil, and a drizzle of tangy balsamic glaze.', 'Ingredients': 'Toasted bread, fresh mozzarella, tomatoes, basil, balsamic glaze', 'Price': '$5'}, {'Name': 'Ravioli di Mare', 'Description': 'Delight in homemade ravioli filled with a blend of seafood, bathed in a delicate tomato cream sauce.', 'Ingredients': 'Homemade ravioli, seafood medley, tomato cream sauce', 'Price': '$2'}, {'Name': 'Tiramisu Tartufo', 'Description': 'Experience the decadence of tiramisu transformed into a luxurious truffle, dusted with a hint of cocoa powder.', 'Ingredients': 'Tiramisu dessert, rich truffle, cocoa powder', 'Price': '$9'},

In [10]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Brainstorm creative dish names for Italian food based on popular Italian dishes and ingredients.
{
1. "Dish Name": "Pesto Pollo Pasta",
   "Description": "Tender chicken breast served over al dente pasta, tossed in a flavorful basil pesto sauce.",
2. "Dish Name": "Caprese Bruschetta",
   "Description": "Toasted bread topped with fresh mozzarella, juicy tomatoes, basil, and a drizzle of balsamic glaze.",
3. "Dish Name": "Ravioli di Mare",
   "Description": "Homemade ravioli stuffed with a medley of seafood, served in a light tomato cream sauce.",
4. "Dish Name": "Tiramisu Tartufo",
   "Description": "Decadent tiramisu dessert in t

# Approach 2: Step through the process yourself as the Meta Agent
- You can use agents manually using `use_agent(agent_name: str, agent_task: str)` The outputs of the agents will be stored into `subtasks_completed` of the meta agent automatically
- You can also interact with the meta agent using `reply_user(task: str, stateful: bool = False)`. This generates a reply for the agent to the `task` using `subtasks_completed` as context. If `stateful` is true, this will also store the output into `subtasks_completed`
- If `subtasks_completed` is too lengthy, you can run `summarise_subtasks_completed(task: str)` This summarises `subtasks_completed` according to what is relevant to the `task`
- If you want to redo a subtask, you can remove the most recent subtask from `subtasks_completed` using `remove_last_subtask()`

In [24]:
# reset the state of the agent
my_agent.reset()
# currently should have no task and nothing in subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [25]:
# Assign a task for the meta agent - this helps to contextualise what you give each of the inner agents later on
my_agent.assign_task('Give me 5 menu items with name, description, ingredients and price based on Italian food choices.')

In [26]:
# This will run the Agent Creative Writer (with shared subtasks_completed as meta agent)
# Output of task will go directly into subtasks_completed - we view the inner agent as simply an extension of the meta agent
my_agent.use_agent('Creative Writer', 'Generate 5 Italian dish names and descriptions')

Calling function Creative Writer with parameters {'instruction': 'Generate 5 Italian dish names and descriptions'}

### Start of Inner Agent: Creative Writer ###
Subtask identified: Brainstorm 5 Italian dish names
Getting LLM to perform the following task: Brainstorm 5 Italian dish names
> {
1. "Pesto Paradise Pasta": A delightful blend of al dente pasta coated in homemade basil pesto sauce, topped with pine nuts and Parmesan cheese.
2. "Mamma Mia Meatballs": Juicy meatballs simmered in rich tomato sauce, served with a side of spaghetti.
3. "Caprese Capriccio": A refreshing salad of ripe tomatoes, fresh mozzarella, basil leaves, drizzled with balsamic glaze.
4. "Ravioli Rendezvous": Handmade ravioli stuffed with creamy ricotta and spinach, finished with a savory marinara sauce.
5. "Tiramisu Temptation": Layers of espresso-soaked ladyfingers and mascarpone cheese, dusted with cocoa powder."}

Task completed successfully!

### End of Inner Agent: Creative Writer ###

> {'Status': 'Comple

In [27]:
# subtasks_completed should be updated with the Creative Agent's outputs
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Brainstorm 5 Italian dish names
{
1. "Pesto Paradise Pasta": A delightful blend of al dente pasta coated in homemade basil pesto sauce, topped with pine nuts and Parmesan cheese.
2. "Mamma Mia Meatballs": Juicy meatballs simmered in rich tomato sauce, served with a side of spaghetti.
3. "Caprese Capriccio": A refreshing salad of ripe tomatoes, fresh mozzarella, basil leaves, drizzled with balsamic glaze.
4. "Ravioli Rendezvous": Handmade ravioli stuffed with creamy ricotta and spinach, finished with a savory marinara sauce.
5. "Tiramisu Temptation": Layers of espresso-soaked ladyfingers and mascarpone cheese, dusted with cocoa po

In [28]:
my_agent.use_agent('Chef', 'Generate ingredients for the 5 Italian dishes')

Calling function Chef with parameters {'instruction': 'Generate ingredients for the 5 Italian dishes'}

### Start of Inner Agent: Chef ###
Subtask identified: Create a list of ingredients for "Pesto Paradise Pasta"
Getting LLM to perform the following task: Create a list of ingredients for "Pesto Paradise Pasta"
> ['Pasta', 'Basil', 'Pine nuts', 'Parmesan cheese']

Subtask identified: Create a list of ingredients for "Mamma Mia Meatballs"
Getting LLM to perform the following task: Create a list of ingredients for "Mamma Mia Meatballs"
> Ingredients for "Mamma Mia Meatballs": Ground beef, breadcrumbs, egg, garlic, onion, parsley, salt, pepper, tomato sauce, spaghetti.

Subtask identified: Create a list of ingredients for "Caprese Capriccio"
Getting LLM to perform the following task: Create a list of ingredients for "Caprese Capriccio"
> Ingredients for "Caprese Capriccio": Tomatoes, fresh mozzarella, basil leaves, balsamic glaze.

Subtask identified: Create a list of ingredients for "Ra

In [29]:
my_agent.use_agent('Economist', 'Generate prices for the 5 Italian dishes')

Calling function Economist with parameters {'instruction': 'Generate prices for the 5 Italian dishes'}

### Start of Inner Agent: Economist ###
Subtask identified: Generate prices for "Pesto Paradise Pasta"
Calling function dish_price with parameters {'list_of_dish_names': ['Pesto Paradise Pasta']}
> {'Price': {'Pesto Paradise Pasta': '$1'}}

Subtask identified: Generate prices for "Mamma Mia Meatballs"
Calling function dish_price with parameters {'list_of_dish_names': ['Mamma Mia Meatballs']}
> {'Price': {'Mamma Mia Meatballs': '$7'}}

Subtask identified: Generate prices for "Caprese Capriccio"
Calling function dish_price with parameters {'list_of_dish_names': ['Caprese Capriccio']}
> {'Price': {'Caprese Capriccio': '$3'}}

Subtask identified: Generate prices for "Ravioli Rendezvous"
Calling function dish_price with parameters {'list_of_dish_names': ['Ravioli Rendezvous']}
> {'Price': {'Ravioli Rendezvous': '$5'}}

Subtask identified: Generate prices for "Tiramisu Temptation"
Calling 

In [30]:
my_agent.use_agent('Boss', 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay and explain why')

Calling function Boss with parameters {'instruction': 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay and explain why'}

### Start of Inner Agent: Boss ###
Task completed successfully!

### End of Inner Agent: Boss ###

> {'Status': 'Completed'}



In [31]:
# Get a targeted response from your agent, and add this to subtask_completed
output = my_agent.reply_user('Choose the 3 menu items for Gordon Ramsay line by line in this format - Dish Name, Dish Description, Dish Ingredients, Dish Price')

Pesto Paradise Pasta, A delightful blend of al dente pasta coated in homemade basil pesto sauce, topped with pine nuts and Parmesan cheese, Ingredients: Pasta, Basil, Pine nuts, Parmesan cheese, Price: $1
Mamma Mia Meatballs, Juicy meatballs simmered in rich tomato sauce, served with a side of spaghetti, Ingredients: Ground beef, breadcrumbs, egg, garlic, onion, parsley, salt, pepper, tomato sauce, spaghetti, Price: $7
Tiramisu Temptation, Layers of espresso-soaked ladyfingers and mascarpone cheese, dusted with cocoa powder, Ingredients: Ladyfingers, espresso, mascarpone cheese, cocoa powder, Price: $6


In [32]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Brainstorm 5 Italian dish names
{
1. "Pesto Paradise Pasta": A delightful blend of al dente pasta coated in homemade basil pesto sauce, topped with pine nuts and Parmesan cheese.
2. "Mamma Mia Meatballs": Juicy meatballs simmered in rich tomato sauce, served with a side of spaghetti.
3. "Caprese Capriccio": A refreshing salad of ripe tomatoes, fresh mozzarella, basil leaves, drizzled with balsamic glaze.
4. "Ravioli Rendezvous": Handmade ravioli stuffed with creamy ricotta and spinach, finished with a savory marinara sauce.
5. "Tiramisu Temptation": Layers of espresso-soaked ladyfingers and mascarpone cheese, dusted with cocoa po

In [33]:
# you can summarise the subtasks_history for the next task
my_agent.summarise_subtasks_completed('Generate the 3 Italian dishes for Gordon Ramsay in menu format')

Pesto Paradise Pasta, A delightful blend of al dente pasta coated in homemade basil pesto sauce, topped with pine nuts and Parmesan cheese, Ingredients: Pasta, Basil, Pine nuts, Parmesan cheese, Price: $1
Mamma Mia Meatballs, Juicy meatballs simmered in rich tomato sauce, served with a side of spaghetti, Ingredients: Ground beef, breadcrumbs, egg, garlic, onion, parsley, salt, pepper, tomato sauce, spaghetti, Price: $7
Tiramisu Temptation, Layers of espresso-soaked ladyfingers and mascarpone cheese, dusted with cocoa powder, Ingredients: Ladyfingers, espresso, mascarpone cheese, cocoa powder, Price: $6


In [34]:
# subtask has been summarised, helping to reduce the context significantly for later task generation
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Summary of Generate the 3 Italian dishes for Gordon Ramsay in menu format
Pesto Paradise Pasta, A delightful blend of al dente pasta coated in homemade basil pesto sauce, topped with pine nuts and Parmesan cheese, Ingredients: Pasta, Basil, Pine nuts, Parmesan cheese, Price: $1
Mamma Mia Meatballs, Juicy meatballs simmered in rich tomato sauce, served with a side of spaghetti, Ingredients: Ground beef, breadcrumbs, egg, garlic, onion, parsley, salt, pepper, tomato sauce, spaghetti, Price: $7
Tiramisu Temptation, Layers of espresso-soaked ladyfingers and mascarpone cheese, dusted with cocoa powder, Ingredients: Ladyfingers, espres

In [35]:
# we also allow you to manually remove the last item added to subtasks_completed, so you can revert your agent state if needed for backtracking
my_agent.remove_last_subtask()

Removed last subtask from subtasks_completed: ('Summary of Generate the 3 Italian dishes for Gordon Ramsay in menu format', 'Pesto Paradise Pasta, A delightful blend of al dente pasta coated in homemade basil pesto sauce, topped with pine nuts and Parmesan cheese, Ingredients: Pasta, Basil, Pine nuts, Parmesan cheese, Price: $1\nMamma Mia Meatballs, Juicy meatballs simmered in rich tomato sauce, served with a side of spaghetti, Ingredients: Ground beef, breadcrumbs, egg, garlic, onion, parsley, salt, pepper, tomato sauce, spaghetti, Price: $7\nTiramisu Temptation, Layers of espresso-soaked ladyfingers and mascarpone cheese, dusted with cocoa powder, Ingredients: Ladyfingers, espresso, mascarpone cheese, cocoa powder, Price: $6')


In [36]:
# visualise your agent's status - should have empty subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed: None
Is Task Completed: False
